In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Installing Libraries

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
%%bash

%%bash

pip install --upgrade pip
pip install farm-haystack[all]

In [4]:
# Here are some imports that we'll need

from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore, FAISSDocumentStore

In [5]:
# Here are some imports that we'll need
import logging
import os, shutil

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Testing

In [8]:
import os, shutil
from haystack.schema import Document
from haystack.nodes.file_converter import TextConverter
from haystack.nodes.preprocessor import PreProcessor
from haystack.document_stores import FAISSDocumentStore

doc_dir = "data/text_chunks"

n = 512
m = 384
preprocessor = PreProcessor(
    split_length=n,
    split_overlap=m,
    clean_empty_lines=False,
    split_respect_sentence_boundary=False,
    clean_whitespace=False,
)

file_path = '/content/sample_contract.txt'
with open(file_path, encoding="utf-8", errors="ignore", mode="r") as f:
  text = f.read()
# # converting text to haystack document format
# converter = TextConverter()
# converted_doc = converter.convert(file_path=file_path)[0]
converted_doc = Document(text)
# splitting document into chunks
docs = preprocessor.process([converted_doc])


## OPTIONAL PART ##
try:
  shutil.rmtree(doc_dir)
  print("creating data folder again..")
except:
  print("creating data folder..")

os.makedirs(doc_dir)

chunks = [k.content for k in docs]
for chunk_id in range(len(chunks)):
  text_file_path = f'{doc_dir}/{chunk_id}.txt'
  with open(text_file_path, 'w') as f:
      f.write(chunks[chunk_id])

Preprocessing: 100%|██████████| 1/1 [00:00<00:00,  7.70docs/s]

creating data folder..


In [9]:
len(docs)

53

In [10]:
####
# initializing document store
document_store = FAISSDocumentStore(similarity="dot_product")
# write the dicts containing documents to our DB.
document_store.write_documents(docs)

Writing Documents: 10000it [00:00, 41169.99it/s]


In [11]:
print([len(k.split()) for k in chunks])

[572, 563, 562, 562, 557, 558, 559, 556, 559, 557, 556, 556, 555, 558, 558, 558, 557, 554, 551, 553, 555, 555, 555, 554, 554, 554, 556, 554, 553, 552, 555, 559, 565, 565, 560, 560, 555, 558, 560, 559, 559, 556, 558, 557, 557, 558, 559, 557, 557, 559, 563, 568, 545]


In [12]:
load_dir = "/content/drive/MyDrive/CLM/dpr_models/dpr_models/dpr_model_small_2023-08-02-11:41__BS_4_epochs_10_ncxts_15"
retriever_small = DensePassageRetriever.load(load_dir=load_dir, document_store=document_store)

document_store.update_embeddings(retriever_small)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.nodes.retriever.dense:DPR model loaded from /content/drive/MyDrive/CLM/dpr_models/dpr_models/dpr_model_small_2023-08-02-11:41__BS_4_epochs_10_ncxts_15
INFO:haystack.document_stores.faiss:Updating embeddings for 53 docs...
Create embeddings: 100%|██████████| 64/64 [00:48<00:00,  1.50 Docs/s]
Documents Processed: 10000 docs [00:48, 205.45 docs/s]


In [13]:
# New Questions
clause_question = {
"DocumentName": "What is the name/title of the agreement?",
"EffectiveDate" : "What is the effective date of the agreement? or When does the agreement become operative?",
"ValidityTerm": "Extract the clause depicting the initial validity period of the agreement. Do not include any renewal information.",
"GoverningLaw": "Extract the clause related to the law that governs the interpretation of the contract? (Governing Law)",
"LiquidityDamages": "Extract the clause that would award either party liquidated damages for breach or a fee upon the termination of a contract (termination fee)?",
"IPR": "Extract intellectual property rights complete clauses present in the contract. (Ownership/Intellectual Property Rights)",
"TerminationClause": "Extract all the clauses under which this agreement can be terminated. (Termination/Cancellation)",
"AutoRenewal": "What is the auto renewal term after the initial term expires?",
"TerminationNoticePeriod": "What is the waiting/notice period if a party can terminate this contract with or without cause (solely by giving a notice and allowing a waiting period to expire)? (Termination Notice Period)",
"NonRenewalNoticePeriod": "What is the prior notice required to prevent the renewal of the contract? (Non-Renewable Notice Period)",
"LicenseGrant": "Extract the complete License Grant clause which contains all the information related to license granted by one party to its counterparty from the contract?",
"WarrantyClause": "Extract the complete Warranty clause which contains information about warranty against defects or errors in technology, products, or services provided under the contract?",
"ThirdParty": "Extract the complete clause related to a non-contracting party who is a beneficiary to some or all of the clauses in the contract and therefore can enforce its rights against a contracting party. (Third Party Beneficiary/Sub-Contractor/Re-consignment)",
"LimitationOfLiability": "Extract the complete clause related to a party's liability upon the breach of its obligation in the contract? (Limitation Of Liability)",
"Support": "Extract the clause related one more party offering Support/Maintenance services or goods to another party mentioned in the contract. (if any)",
"ObligationRakuten": "What are the obligations/duties/responsibilities of Rakuten mentioned in the contract.",
"ObligationVendor": "Extract the clause(s) related to the obligations/duties/responsibilities of party other than Rakuten from the contract.",
"Assignment": "Extract the complete clause related to the provision regarding Assignment/Non-Assignment mentioned in the contract?",
"PublicAnnouncement": "Extract the full clause related to publicity or public announcement in the contract.(if any)",
"ForceMajeure": "Extract the full clause related to Force Majeure in the contract. (Force Majeure)",
"Vendor": "Extract the names of one or more parties which signed the contract as vendor (not as the Assigner)?",
"PaymentTerms": "Extract the complete Payment Terms clause which contains information related to Payment/Invoice/Fee?",
"Indmenity": "Extract the complete indemnification clause which contains contractual obligation of one party to indemnify the other party.",
"Currency": "Which country's currency will be used for payment? (USD/JPY/EURO/INR)",
"Update": "Find the clause about updation of services. (Update/Upgrade)",
"DelayedPayment": "Extract the information related to the consequences of delay in payment (Delay/Delinquency Charges)?",
"Exclusions": "Extract complete clause related to Exclusions/Anti-Social which seek to prevent the misuse of services/software.",
"Scope": "Extract the complete Scope/Purpose/Entire Agreement clause from this contract?",
"SLA": "Extract complete SLA clause.",
"Condition": "Extract the complete clause related to the terms of the agreement (which includes validity term, auto-renewal term and non-renewable notice period) present in the contract with conditions.",
"Contract Value": "What is the total amount of contract?"
}

In [14]:
small_clauses=['DocumentName', 'EffectiveDate', 'ValidityTerm', 'LimitationOfLiability',
               'Vendor', 'Currency', 'Scope', 'AutoRenewal', 'NonRenewalNoticePeriod', 'Condition',
               'GoverningLaw', 'Update', 'Assignment', 'DelayedPayment', 'ThirdParty',
               'PublicAnnouncement', 'LiquidityDamages', 'ForceMajeure', 'TerminationNoticePeriod', 'Contract Value']
long_clauses = ['IPR', 'TerminationClause', 'LicenseGrant', 'WarrantyClause', 'Support',
                'ObligationRakuten', 'ObligationVendor', 'PaymentTerms', 'Exclusions','Indmenity']

In [ ]:
candidate_documents = retriever_small.retrieve(
    query=clause_question["EffectiveDate"],
    top_k=3
)

doc_dict = dict()
for doc in docs:
    doc_dict[doc.meta["_split_id"]] = doc

ll = [i+1 for i in range(len(candidate_documents))]
dict(zip(ll, candidate_documents))

#### Zero-Shot prompt

In [17]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

# Build prompt
prompt_template_zero_shot = """Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. If you don't know the answer or if the answer is not present in the given context, just say "NA", don't try to make up an answer.

Context: {join(documents)}

Question: {query}
Answer:"""

rag_prompt = PromptTemplate(
    prompt=prompt_template_zero_shot,
    output_parser=AnswerParser(),
)

#### Few-Shot Prompt

In [18]:
all_examples = {
    'DocumentName':
     [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['DocumentName']}",
        "answer": "Statement of Work"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['DocumentName']}",
        "answer": "MASTER SOFTWARE LICENSE AND SUPPORT AGREEMENT"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['DocumentName']}",
        "answer": "Confidentiality Agreement"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['DocumentName']}",
        "answer": "Amendment to the interconnection agreement"
        },
    ],
    'EffectiveDate':
    [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['EffectiveDate']}",
        "answer": "June 30, 2021"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['EffectiveDate']}",
        "answer": "24/9/2019"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['EffectiveDate']}",
        "answer": "December 24, 2021"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['EffectiveDate']}",
        "answer": "July 1st, 2020"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['EffectiveDate']}",
        "answer": "NA"
        },
    ],
    'ValidityTerm':
    [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['ValidityTerm']}",
        "answer": "one year"
        },
         {
        "context": """CONTEXT""",
        "question": f"{clause_question['ValidityTerm']}",
        "answer": "five (5) years"
        },
         {
        "context": """CONTEXT""",
        "question": f"{clause_question['ValidityTerm']}",
        "answer": "NA"
        },
    ],
    'Vendor':
    [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['Vendor']}",
        "answer": "NEC Corporation"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['Vendor']}",
        "answer": "Airspan Networks Inc."
        },
    ],
    'Currency':
    [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['Currency']}",
        "answer": "Japanese Yen"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['Currency']}",
        "answer": "US dollars"
        },
        {
        "context": """“CONTEXT""",
        "question": f"{clause_question['Currency']}",
        "answer": "USD"
        },
    ],
    'AutoRenewal':
    [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['AutoRenewal']}",
        "answer": "two (2) year"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['AutoRenewal']}",
        "answer": "an additional year"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['AutoRenewal']}",
        "answer": "NA"
        },
    ],
    'NonRenewalNoticePeriod':
    [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['NonRenewalNoticePeriod']}",
        "answer": "9 months"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['NonRenewalNoticePeriod']}",
        "answer": "forty-five (45) days"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['NonRenewalNoticePeriod']}",
        "answer": "NA"
        },
    ],
    'TerminationNoticePeriod':
    [
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['TerminationNoticePeriod']}",
        "answer": "thirty (30) days"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['TerminationNoticePeriod']}",
        "answer": "60 days"
        },
        {
        "context": """CONTEXT""",
        "question": f"{clause_question['TerminationNoticePeriod']}",
        "answer": "three months"
        },
    ]

}

In [19]:
# Create example prompts

def create_examples(clause = 'NonRenewalNoticePeriod'):
    example_template = """Context: {context}
    Question: {question}
    Answer: {answer}
    """
    examples = "\n".join([str(i+1)+ '. ' + example_template.format(context=example['context'],question=example['question'],answer=example['answer']) for i, example in enumerate(all_examples[clause])])
    return examples

In [20]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

# Build prompt
prefix = """Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. If you don't know the answer or if the answer is not present in the given context, just say "NA", don't try to make up an answer.

Here are some examples:

"""

suffix="""\n\nNow, use the context given below to extract an answer to the question that follows.\n
Context: {join(documents)}

Question: {query}
Answer:"""

prompt_template = prefix + create_examples('NonRenewalNoticePeriod') + suffix

rag_prompt = PromptTemplate(
    prompt=prompt_template,
    output_parser=AnswerParser(),
)

In [21]:
print(prompt_template)

Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. If you don't know the answer or if the answer is not present in the given context, just say "NA", don't try to make up an answer.

Here are some examples:

1. Context: This Agreement shall become effective as of the Effective Date, (if unspecified, the last-occur-date of signing shall be deemed to be Effective Date) and continue in full force and effect for five (5) years. Such term shall be automatically renewed for successive two (2) year term unless either Party delivers a written notice to the other Party indicating such Party's intention not to renew this Agreement 9 months prior to the expiration of the applicable term.
    Question: What is the prior notice required to prevent the renewal of the contract? (Non-Re

In [22]:
# rag_prompt

In [23]:
openai_api_key = os.getenv("OPENAI_API_KEY", None)

prompt_node = PromptNode(
    model_name_or_path="gpt-3.5-turbo-instruct",
    api_key=openai_api_key,
    default_prompt_template=rag_prompt,
    top_k=3,
    max_length=512,
    model_kwargs={'temperature': 0.1}
)

In [24]:
from haystack.pipelines import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever_small, name="Retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="Prompt_node", inputs=["Retriever"])

In [25]:
output = pipe.run(query=clause_question["EffectiveDate"],
                  params={"Retriever": {"top_k": 3}},
                  debug=True)

for o in output["answers"]:
    print(o.answer)

February 20, 2019
February 20, 2019
February 20, 2019


#### Saving Results to CSV

In [ ]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser
from haystack.pipelines import Pipeline

# Build prompt
prompt_template_zero_shot = """Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. If you don't know the answer or if the answer is not present in the given context, just say "NA", don't try to make up an answer.

Context: {join(documents)}

Question: {query}
Answer:"""

# Create a prefix and suffix for the prompt
prefix = """Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. If you don't know the answer or if the answer is not present in the given context, just say "NA", don't try to make up an answer.

Here are some examples:

"""

suffix="""\n\nNow, use the context given below to extract an answer to the question that follows.\n
Context: {join(documents)}

Question: {query}
Answer:"""


import csv
fields = ['Entity/Clause', 'Query', 'Results', 'Few-Shot']
few_shot="No"
with open('results_gpt-3.5-turbo-instruct.csv', mode='w', encoding='utf-8') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(fields)
    for k in small_clauses:
        if k in ['DocumentName','EffectiveDate', 'ValidityTerm', 'Vendor', 'Currency', 'AutoRenewal', 'NonRenewalNoticePeriod', 'TerminationNoticePeriod']:
            # Create the FewShotPromptTemplate
            prompt_template = prefix + create_examples(k) + suffix
            few_shot = "Yes"
        else:
            prompt_template = prompt_template_zero_shot
            few_shot = "No"

        rag_prompt = PromptTemplate(
            prompt=prompt_template,
            output_parser=AnswerParser(),
        )
        query = clause_question[k]
        prompt_node = PromptNode(
            model_name_or_path="gpt-3.5-turbo-instruct",
            api_key=openai_api_key,
            default_prompt_template=rag_prompt,
            top_k=1,
            max_length=512,
            model_kwargs={'temperature': 0.1}
        )
        pipe = Pipeline()
        pipe.add_node(component=retriever_small, name="Retriever", inputs=["Query"])
        pipe.add_node(component=prompt_node, name="Prompt_node", inputs=["Retriever"])
        output = pipe.run(query=clause_question[k],
                        params={"Retriever": {"top_k": 3}},
                        debug=True)
        result = output["answers"][0].answer
        csvwriter.writerows([[k,query,result.lstrip(),few_shot]])
        print(result)